# Analysis Centiment Review Tokopedia

In [1]:
import pandas as pd
import numpy as np
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import pandas as pd
from ibm_watsonx_ai.credentials import Credentials
import time
from dotenv import load_dotenv
import os

In [2]:
# --- IBM Credentials (ISI SESUAI PUNYAMU) ---
load_dotenv()

api_key = os.getenv("API_KEY")
project_id = os.getenv("PROJECT_ID")
region = os.getenv("REGION")
model_id = os.getenv("MODEL_ID")

In [3]:
# --- Setup Credentials Object ---
credentials = Credentials(
    url=f"https://{region}.ml.cloud.ibm.com",
    api_key=api_key
)

# Setup Inference
inference = ModelInference(
    model_id=model_id,
    credentials=credentials,
    project_id=project_id
)

c:\Users\TUF GAMING\.conda\envs\verzha_new\Lib\site-packages\ibm_watsonx_ai\foundation_models\utils\utils.py:436: LifecycleWarning: Model 'ibm/granite-13b-instruct-v2' is in deprecated state from 2025-06-18 until 2025-10-15. IDs of alternative models: ibm/granite-3-3-8b-instruct. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warn(model_state_warning, category=LifecycleWarning)


In [4]:
def classify_sentiment(text):
    prompt = f"""
Tolong klasifikasikan ulasan berikut ke dalam salah satu: Positif, Negatif, atau Netral.

Ulasan:
"{text}"

Jawaban (hanya salah satu dari: Positif, Negatif, Netral):
"""
    params = {
        "max_tokens": 20,
        "temperature": 0.3
    }
    try:
        response = inference.generate_text(prompt, params)

        # Jika response adalah string langsung:
        if isinstance(response, str):
            return response.strip().split('\n')[0]
        
        elif isinstance(response, dict) and 'results' in response:
            return response['results'][0]['generated_text'].strip().split('\n')[0]

        else:
            return f"Unexpected response: {response}"

    except Exception as e:
        return f"Error: {e}"


In [5]:
# Load data
df = pd.read_csv('ulasan_aplikasi.csv')

# Ambil kolom yang dibutuhkan
df = df[['content', 'score']].dropna()

# Buat label manual dari score
def label_manual(score):
    if score >= 4:
        return 'Positif'
    elif score == 3:
        return 'Netral'
    else:
        return 'Negatif'

df['label_manual'] = df['score'].apply(label_manual)

In [6]:
sample_df = df.sample(10, random_state=42).copy()
sample_df['label_granite'] = sample_df['content'].apply(lambda x: classify_sentiment(x))
print(sample_df[['content', 'score', 'label_manual', 'label_granite']])


                                                 content  score label_manual  \
41768  Jangan percaya aplikasi ini kalau ada diskon, ...      1      Negatif   
26298  Sangat sangat membantu,, beli kelas pelatihan ...      5      Positif   
67265  Aplikasi bodohhhh disusahin banget, mau buka t...      1      Negatif   
23766  Aplikasinya berat, suka refresh sendiri,lagi m...      1      Negatif   
47094  Oii.. Apaan sih. Bentar2 belanja auto cancel. ...      1      Negatif   
28556  Aku pake app ini cm cek hrga, sbg saran kurir ...      3       Netral   
39786               bagus banget.. banyain promo dong 😁😁      5      Positif   
49325  👍😊 beli kursus online.. Sekali klik langsung l...      5      Positif   
8420   keuntungan menggunakan Plus apa? hanya mendapa...      1      Negatif   
36800  Mentang2 ongkir gratis, pengiriman jabodetabek...      1      Negatif   

      label_granite  
41768      Negative  
26298        Netral  
67265        Netral  
23766        Netral  
47094    

In [7]:
result = classify_sentiment("Aplikasinya sangat lambat dan bikin kesel")
print("Hasil prediksi:", result)

Hasil prediksi: Netral
